In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import glob
import cv2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps
!pip install -q "/kaggle/input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install -q "/kaggle/input/hpapytorchzoozip/pytorch_zoo-master"
!pip install -q "/kaggle/input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"

#!wget "https://www.kaggleusercontent.com/kf/60257229/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..CiJyNbJXntOyTTiedEu0RA.9ehsk6hfDsNYbo2hw344h9FJN_gNQ_7JbmG5addEB5yMS2GEVsiYEbCYtTZrl_GWl2rm_BlEfgGn-uvzr1m1MsAgen97qAVX-FG6bCguUWThX4Sj1YbOtZYUO_bdvn63ce9n18_BC5CB2BfcmjCXIEamsaM-LF0PlOmlJ42WSrnyX7A3sUlBAV8AXgaeku0F17F3qEGClYYro9kQPAgT4Wg2zQVegDNtpE_52kj0mOVagAoWPw5N8sD2ueY4HrY4uTgNvQwHKQq60ce3Wu0XVODROcpZZSDIpfarofwUZQhSe4GmnoxHJA4kBN_kG1_0YdnsvOKCVmLoVC7BmChqPr-9llQhyT6pBVRIhCBXanH2dYHy-bLxZPXXmH-bYgVuxNTt3wKR-LcVScafukQfwg_tHBOAxb5U7a7ZuV7zfDY5HjwMrhjdd0JkerXrBCvOhsC5NQ4H9eIUB0csEZhQwEQmGk6u9w0NvDS-7gdz71b-qsI7Atk4TNTa9yEoJbgXZH6kRRHlb9kpjA3XCPcnV5j0Mwjli-qdqJj_uXwd66CLpy9wvnRaniqm8bOlf7wVfGCXZqdEjFp1WlYcWuoKKdsMZv-6-SVXA52_suOaMq1pf7HodgDobeohQBGZQuzLv_xRZnOUZPlwAE0OZa_vB9W7a48mwPWnS6Lh0llFaHk.3bLf-RBpLBwqm6-8VMCrwA/model_green.h5"

In [3]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
import glob
import tensorflow_addons as tfa

In [4]:
ONLY_PUBLIC = False

In [5]:
import base64
from pycocotools import mask as mutils
from pycocotools import _mask as coco_mask
import typing as t
import zlib
def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str.decode() + ' '

In [6]:
import matplotlib.pyplot as plt
import cv2

In [7]:
def decreasing_non_label(pred):
  '''
  expect shape(dls.c,)
  '''
  #pred = pred.numpy()
  avg = torch.mean(pred)
  
  pred[pred<avg] -= 4
  
  return pred


In [8]:
import tensorflow as tf
import efficientnet.tfkeras as efn
from tqdm import tqdm

In [9]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
def enc2mask(encs, shape):
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for m,enc in enumerate(encs):
        if isinstance(enc,np.float) and np.isnan(enc): continue
        s = enc.split()
        for i in range(len(s)//2):
            start = int(s[2*i]) - 1
            length = int(s[2*i+1])
            img[start:start+length] = 1 + m
    return img.reshape(shape).T

In [10]:
import sys

In [11]:
from tensorflow import keras

In [12]:
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [13]:
import os.path
import urllib
import zipfile

import numpy as np
import scipy.ndimage as ndi
from skimage import filters, measure, segmentation
from skimage.morphology import (binary_erosion, closing, disk,
                                remove_small_holes, remove_small_objects)
import torch
HIGH_THRESHOLD = 0.4
LOW_THRESHOLD = HIGH_THRESHOLD - 0.25


def download_with_url(url_string, file_path, unzip=False):
    """Download file with a link."""
    with urllib.request.urlopen(url_string) as response, open(
        file_path, "wb"
    ) as out_file:
        data = response.read()  # a `bytes` object
        out_file.write(data)

    if unzip:
        with zipfile.ZipFile(file_path, "r") as zip_ref:
            zip_ref.extractall(os.path.dirname(file_path))


def __fill_holes(image):
    """Fill_holes for labelled image, with a unique number."""
    boundaries = segmentation.find_boundaries(image)
    image = np.multiply(image, np.invert(boundaries))
    image = ndi.binary_fill_holes(image > 0)
    image = ndi.label(image)[0]
    return image





def label_cell2(nuclei_pred, cell_pred):
    """Label the cells and the nuclei.
    Keyword arguments:
    nuclei_pred -- a 3D numpy array of a prediction from a nuclei image.
    cell_pred -- a 3D numpy array of a prediction from a cell image.
    Returns:
    A tuple containing:
    nuclei-label -- A nuclei mask data array.
    cell-label  -- A cell mask data array.
    0's in the data arrays indicate background while a continous
    strech of a specific number indicates the area for a specific
    cell.
    The same value in cell mask and nuclei mask refers to the identical cell.
    NOTE: The nuclei labeling from this function will be sligthly
    different from the values in :func:`label_nuclei` as this version
    will use information from the cell-predictions to make better
    estimates.
    """
    def __wsh(
        mask_img,
        threshold,
        border_img,
        seeds,
        threshold_adjustment=0.35,
        small_object_size_cutoff=10,
    ):
        img_copy = np.copy(mask_img)
        m = seeds * border_img  # * dt
        img_copy[m <= threshold + threshold_adjustment] = 0
        img_copy[m > threshold + threshold_adjustment] = 1
        img_copy = img_copy.astype(np.bool)
        img_copy = remove_small_objects(img_copy, small_object_size_cutoff).astype(
            np.uint8
        )

        mask_img[mask_img <= threshold] = 0
        mask_img[mask_img > threshold] = 1
        mask_img = mask_img.astype(np.bool)
        mask_img = remove_small_holes(mask_img, 63)
        mask_img = remove_small_objects(mask_img, 1).astype(np.uint8)
        markers = ndi.label(img_copy, output=np.uint32)[0]
        labeled_array = segmentation.watershed(
            mask_img, markers, mask=mask_img, watershed_line=True
        )
        return labeled_array

    nuclei_label = __wsh(
        nuclei_pred[..., 2] / 255.0,
        0.4,
        1 - (nuclei_pred[..., 1] + cell_pred[..., 1]) / 255.0 > 0.05,
        nuclei_pred[..., 2] / 255,
        threshold_adjustment=-0.25,
        small_object_size_cutoff=32,
    )

    # for hpa_image, to remove the small pseduo nuclei
    nuclei_label = remove_small_objects(nuclei_label, 157)
    nuclei_label = measure.label(nuclei_label)
    # this is to remove the cell borders' signal from cell mask.
    # could use np.logical_and with some revision, to replace this func.
    # Tuned for segmentation hpa images
    threshold_value = max(0.22, filters.threshold_otsu(cell_pred[..., 2] / 255) * 0.5)
    # exclude the green area first
    cell_region = np.multiply(
        cell_pred[..., 2] / 255 > threshold_value,
        np.invert(np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8)),
    )
    sk = np.asarray(cell_region, dtype=np.int8)
    distance = np.clip(cell_pred[..., 2], 255 * threshold_value, cell_pred[..., 2])
    cell_label = segmentation.watershed(-distance, nuclei_label, mask=sk)
    cell_label = remove_small_objects(cell_label, 344).astype(np.uint8)
    selem = disk(2)
    cell_label = closing(cell_label, selem)
    cell_label = __fill_holes(cell_label)
    # this part is to use green channel, and extend cell label to green channel
    # benefit is to exclude cells clear on border but without nucleus
    sk = np.asarray(
        np.add(
            np.asarray(cell_label > 0, dtype=np.int8),
            np.asarray(cell_pred[..., 1] / 255 > 0.05, dtype=np.int8),
        )
        > 0,
        dtype=np.int8,
    )
    cell_label = segmentation.watershed(-distance, cell_label, mask=sk)
    cell_label = __fill_holes(cell_label)
    cell_label = np.asarray(cell_label > 0, dtype=np.uint8)
    cell_label = measure.label(cell_label)
    cell_label = remove_small_objects(cell_label, 344) # was 344
    cell_label = measure.label(cell_label)
    cell_label = np.asarray(cell_label, dtype=np.uint16)
    #nuclei_label = np.multiply(cell_label > 0, nuclei_label) > 0
    #nuclei_label = measure.label(nuclei_label)
    #nuclei_label = remove_small_objects(nuclei_label, 157)
    #nuclei_label = np.multiply(cell_label, nuclei_label > 0)

    return cell_label#nuclei_label, cell_label

In [14]:
!mkdir cell_tile
cell_dir = 'cell_tile'

In [15]:
NUC_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth"
CELL_MODEL = "../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth"
segmentator = cellsegmentator.CellSegmentator(
        NUC_MODEL,
        CELL_MODEL,
        scale_factor=0.25,
        device="cuda",
        padding=True,
        multi_channel_model=True,
    )
i_df = pd.read_csv("../input/hpa-single-cell-image-classification/sample_submission.csv")
if ONLY_PUBLIC==True:
    i_df = pd.read_csv("../input/hpa-sample-submission-with-extra-metadata/updated_sample_submission.csv")
bs = 40
bs_now=0
image_list=[]
mt = []
er = []
nu = []
ID_list=[]
for i in tqdm(i_df.index):
    ID = i_df.loc[i,'ID']
    #if len(i_df)==559:
    #    continue
    if len(glob.glob('../input/cell-tile-fast-cegmentor/cell_tile/'+ID+'.npz'))>0:
        continue
    W= int(i_df[i_df['ID']==ID]['ImageWidth'].values[0])
    H= int(i_df[i_df['ID']==ID]['ImageHeight'].values[0])
    ID_list.append(ID)
    name = '../input/hpa-single-cell-image-classification/test' + '/' +ID+ '_red.png'
    mt.append(name)
    er.append(name.replace('red', 'yellow'))
    nu.append(name.replace('red', 'blue'))
    images = [mt, er, nu]
    bs_now+=1
    if bs_now>=bs:
        
        # For nuclei
        nuc_segmentations = segmentator.pred_nuclei(images[2])

        # For full cells
        cell_segmentations = segmentator.pred_cells(images)

        
        
        
        cell_mask=[]
        for j, pred in enumerate(cell_segmentations):
            size = cell_segmentations[j].shape[0]
            cell_mask_ = label_cell2(cv2.resize(nuc_segmentations[j],(512,512)), cv2.resize(cell_segmentations[j],(512,512)))
            cell_mask_ = cv2.resize(cell_mask_,(size,size),interpolation=cv2.INTER_NEAREST)
            cell_mask.append(cell_mask_)
        for id_,mask in zip(ID_list,cell_mask):
            np.savez_compressed(f'{cell_dir}/{id_}', mask)
        
        # clear
        
        image_list=[]
        mt = []
        er = []
        nu = []
        ID_list=[]
        bs_now=0
if bs_now>0:
        nuc_segmentations = segmentator.pred_nuclei(images[2])

        # For full cells
        cell_segmentations = segmentator.pred_cells(images)

        
        
        cell_mask=[]
        for j, pred in enumerate(cell_segmentations):
            size = cell_segmentations[j].shape[0]
            cell_mask_ = label_cell2(cv2.resize(nuc_segmentations[j],(512,512)), cv2.resize(cell_segmentations[j],(512,512)))
            cell_mask_ = cv2.resize(cell_mask_,(size,size),interpolation=cv2.INTER_NEAREST)
            cell_mask.append(cell_mask_)
        for id_,mask in zip(ID_list,cell_mask):
            np.savez_compressed(f'{cell_dir}/{id_}', mask)
        
        # clear
        
        image_list=[]
        mt = []
        er = []
        nu = []
        ID_list=[]
        bs_now=0

please compile abn


/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'pytorch_zoo.unet.DPNUnet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set

In [16]:
import gc
del CELL_MODEL
del NUC_MODEL
del cellsegmentator
#del NUC
gc.collect()
torch.cuda.empty_cache()

In [17]:
'''
try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("Running on TPU:", tpu.master())
except ValueError:
            strategy = tf.distribute.get_strategy()
            print(f"Running on {strategy.num_replicas_in_sync} replicas")
'''


'''
model = tf.keras.models.load_model(
            '../input/hpa-classification-efnb7-train/model_green.h5',compile = False
        )
'''

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "... Physical GPUs,", len(logical_gpus), "Logical GPUs ...\n")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)


model = tf.keras.models.load_model(
            '../input/download-model/model_RGB_0.0502.h5'
        )

model_4 = tf.keras.models.load_model(
            '../input/hpa-training-tpu-pseude-label-t/model_RGB_0.0635.h5'
        )
model_2 = tf.keras.models.load_model(
            '../input/atlasmodel/model_RGB.h5'
        )
    
model_3 = tf.keras.models.load_model('../input/hpa-training-tpu/model_RGB_0.2217.h5')
#model_4 = tf.keras.models.load_model('../input/atlasmodel/model_RGB_0.1139.h5')
model_g = tf.keras.models.load_model('../input/fork-of-hpa-training-tpu-2/model_RGB_0.2440.h5')

model_whole = tf.keras.models.load_model(
            '../input/hpa-classification-efnb7-train-tfrec/model_green.h5'
        )

model_whole_2 = tf.keras.models.load_model(
            '../input/atlasmodel/model_green.h5'
        )

model_whole_3 = tf.keras.models.load_model(
            '../input/download-model3/model_green.h5'
        )
# model_whole_4 = tf.keras.models.load_model(
#             '../input/hpa-classification-efnb7-train-noisy/model_green.h5'
#         )


1 ... Physical GPUs, 1 Logical GPUs ...



In [18]:
def pred_string(ID, high, width):
    

    ID = ID
    W = width
    H = high
    

    green = plt.imread('../input/hpa-single-cell-image-classification/test/'+ID+'_green.png')
    red = plt.imread('../input/hpa-single-cell-image-classification/test/'+ID+'_red.png')
    blue = plt.imread('../input/hpa-single-cell-image-classification/test/'+ID+'_blue.png')
    green = green[:,:,np.newaxis]
    blue = blue[:,:,np.newaxis]
    red = red[:,:,np.newaxis]
    all_ = np.concatenate((red,green,blue),axis=-1)
    w = width
    h = high
    mt = glob.glob('../input/hpa-single-cell-image-classification/test' + '/' +ID+ '_red.png')
    er = [f.replace('red', 'yellow') for f in mt]
    nu = [f.replace('red', 'blue') for f in mt]
    images = [mt, er, nu]
    
    #ensemble the image-wise
    all_whole = np.concatenate((green,green,green),axis=-1)
    all_whole_720 = cv2.resize(all_whole,(720,720),
                              interpolation = cv2.INTER_AREA)
    all_whole_720 = np.expand_dims(all_whole_720,axis=0)
    all_whole_512 = cv2.resize(all_whole,(512,512),
                              interpolation = cv2.INTER_AREA)
    all_whole_512 = np.expand_dims(all_whole_512,axis=0)
    all_whole = cv2.resize(all_whole,(600,600),
                              interpolation = cv2.INTER_AREA)
    
    all_whole = np.expand_dims(all_whole,axis=0)
    pred_whole = model_whole.predict(all_whole)
    pred_whole2 = model_whole_2.predict(all_whole)
    pred_whole3 = model_whole_3.predict(all_whole_720)
    #pred_whole4 = model_whole_4.predict(all_whole)
    pred_whole = pred_whole+pred_whole2+pred_whole3
    # For nuclei
    #nuc_segmentations = segmentator.pred_nuclei(images[2])

    # For full cells
    #cell_segmentations = segmentator.pred_cells(images)

    # post-processing
    
    
    #nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
    #FOVname = os.path.basename(mt[i]).replace('red','predictedmask')
    #imageio.imwrite(os.path.join(save_dir,FOVname), cell_mask) 
    file_ =glob.glob('../input/cell-tile-fast-cegmentor/cell_tile/'+ID+'.npz')
    if len(file_)>0:
        cell_mask = np.load(file_[0])['arr_0']
    else:
        cell_mask = np.load(f'{cell_dir}/{ID}.npz')['arr_0']
    pred_strs = []
    for i in range(1,cell_mask.max()+1):
          x,y=(cell_mask==i).nonzero()
          cell_mask_s = cell_mask==i
          cell_mask_s_ = cell_mask_s[:,:,np.newaxis]
          i_c = cell_mask_s_ * all_
          
            
            
            
#           i_c_720 = cv2.resize(i_c,(720,720),
#                               interpolation = cv2.INTER_AREA)
#           i_c_720 = np.stack([i_c_720[:,:,1],i_c_720[:,:,1],i_c_720[:,:,1]],axis=2)
#           i_c_720 = np.expand_dims(i_c_720,axis=0)  
          
          i_c = cv2.resize(i_c,(600,600),
                              interpolation = cv2.INTER_AREA)
          i_c = np.stack([i_c[:,:,1],i_c[:,:,1],i_c[:,:,1]],axis=2)
          i_c = np.expand_dims(i_c,axis=0)
          
            
            
        
          pred_ic = model_whole_2.predict(i_c)
          #pred_ic2 = model_whole.predict(i_c)
          #pred_ic3 = model_whole_3.predict(i_c_720)
          #print(pred_ic)
          lenx=x.max()-x.min()+1
          leny=y.max()-y.min()+1
          xx=x.min()
          yy=y.min()

          interest=cell_mask[xx:xx+lenx,yy:yy+leny]
          all_g=all_[xx:xx+lenx,yy:yy+leny]
          if lenx>leny:
              pad=lenx-leny
              interest=np.pad(interest,((0,0),(pad//2,pad-pad//2)), 'constant', constant_values=(0))
              all_g=np.pad(all_g,((0,0),(pad//2,pad-pad//2),(0,0)), 'constant', constant_values=(0))
          elif leny>lenx:
              pad=leny-lenx
              interest=np.pad(interest,((pad//2,pad-pad//2),(0,0)), 'constant', constant_values=(0))
              all_g=np.pad(all_g,((pad//2,pad-pad//2),(0,0),(0,0)), 'constant', constant_values=(0))
          interest = interest[:,:,np.newaxis]
          interest=(interest==i)

          all_g=interest*all_g

          
          

          




          all_g = cv2.resize(all_g,(256,256),
                              interpolation = cv2.INTER_AREA)
          #all_g = np.transpose(all_g,(2,0,1))
          all_green = np.stack([all_g[:,:,1],all_g[:,:,1],all_g[:,:,1]],axis=2)
          all_g = np.expand_dims(all_g,axis=0)
          all_green = np.expand_dims(all_green,axis=0)
          #all_g=(torch.from_numpy(all_g)).to(device)
          #model.eval()
          pred_c  = 0

          
          pred = model.predict(all_g)
          pred2 = model_2.predict(all_g)
          pred3 = model_3.predict(all_g)
          pred4 = model_4.predict(all_g)
          #pred4 = model_4.predict(all_g)
          predg = model_g.predict(all_green)
          pred = pred+pred2+pred3+pred4+predg+pred_ic
          #pred2 = model_2.predict(all_g)
          #pred3 = model_3.predict(all_g)
          #pred4 = model_4.predict(all_g)
          #pred4 = model_4.predict(all_g)
          #predg = model_g.predict(all_green)
          #pred = (pred+pred_ic)/2.
          # Only get the argmax one
          pred = pred + pred_whole
          pred_c = pred[0]
          #pred_c = decreasing_non_label(pred_c)
            
          rle = encode_binary_mask(cell_mask_s)
          
            
          #set non-green to class 18
          '''
          green_channel = all_g[:,:,1]
          green_channel[green_channel<0.1]=0.
          if np.sum(green_channel)<0.5:
                pred_c = [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,1.0]
          '''
          
          
          for class_id in range(19):
            #cnf = torch.sigmoid(pred_c[class_id])
            cnf = pred_c[class_id]
            pred_strs.append(f'{class_id} {cnf} {rle}{xx+(lenx-1)/2} {yy+(leny-1)/2} ')
            
    return ''.join(pred_strs)[:-1]

In [19]:
pred_string('0040581b-f1f2-4fbe-b043-b6bfea5404bb',2048,2048)

'0 3.9748129844665527 eNqtlT9vhSAQwL8StGfj8AYHB18g9JoQH4MDA2mxNe/7j8WKJqiAxrLc7/5wwHEBRPwkAJbYJ1Xv6kNEyZCa/A18QTHRrQ4l8EmWzSSJaoVmaKkGVHEyYB+iY91IJkEWuodQzI6kEjQAjoS9s6VoANsKybqRTIJkIVuBDL8pFpggXfhjcyDBuKpfjTs75nu8Gp/Ls/ZL5gptX21PFVcx0j+0CqdvJNlfl5d8t0wbu9c35V7bgafzrO2QyZuT8M/1Pts3R/vv6Ppn+wzv7qGQb+4pGO4mRqqnPNMfPH2uRfWw3CPJ6BHZABwqS1Q2kNz/7D96T/O+t40Ku/Hr/Ivu4zSpQkdZQ6g3of9WhX73vwjtf4zh60l/AWpLehU= 121.5 697.5 1 0.03422684594988823 eNqtlT9vhSAQwL8StGfj8AYHB18g9JoQH4MDA2mxNe/7j8WKJqiAxrLc7/5wwHEBRPwkAJbYJ1Xv6kNEyZCa/A18QTHRrQ4l8EmWzSSJaoVmaKkGVHEyYB+iY91IJkEWuodQzI6kEjQAjoS9s6VoANsKybqRTIJkIVuBDL8pFpggXfhjcyDBuKpfjTs75nu8Gp/Ls/ZL5gptX21PFVcx0j+0CqdvJNlfl5d8t0wbu9c35V7bgafzrO2QyZuT8M/1Pts3R/vv6Ppn+wzv7qGQb+4pGO4mRqqnPNMfPH2uRfWw3CPJ6BHZABwqS1Q2kNz/7D96T/O+t40Ku/Hr/Ivu4zSpQkdZQ6g3of9WhX73vwjtf4zh60l/AWpLehU= 121.5 697.5 2 1.9897958040237427 eNqtlT9vhSAQwL8StGfj8AYHB18g9JoQH4MDA2mxNe/7j8WKJqiAxrLc7/5wwHEBRPwkAJbYJ1Xv6kNEyZCa/A18QTHRrQ4l8EmWzSSJaoVmaKkGVHEyYB+iY91IJkEWuodQzI6kEjQAjoS9s6VoA

In [20]:
df_imlist = pd.read_csv('../input/hpa-single-cell-image-classification/sample_submission.csv')
df_imlist

,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
1,004a270d-34a2-4d60-bbe4-365fca868193,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
2,00537262-883c-4b37-a3a1-a4931b6faea5,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
3,00c9a1c9-2f06-476f-8b0d-6d01032874a2,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 1 eNoLCAgIsAQABJ4Beg==
...,...,...,...,...
554,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,1728,1728,0 1 eNoLCAjJNgIABNkBkg==
555,feb955db-6c07-4717-a98b-92236c8e01d8,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
556,fefb9bb7-934a-40d1-8d2f-210265857388,2048,2048,0 1 eNoLCAgIMAEABJkBdQ==
557,ff069fa2-d948-408e-91b3-034cfea428d1,3072,3072,0 1 eNoLCAgIsAQABJ4Beg==


In [21]:
if len(df_imlist) == 558:
    debug = True
    df_imlist = df_imlist[:3]
else:
    debug = False

In [22]:
import time 
start = time.time()

df_imlist['PredictionString'] =  df_imlist.apply(
    lambda row: pred_string(row['ID'],row['ImageHeight'],row['ImageWidth']),
    axis=1
)




end = time.time()

print((end - start)/559)

11.03809729628998


In [23]:
df_imlist

,ID,ImageWidth,ImageHeight,PredictionString
0,0040581b-f1f2-4fbe-b043-b6bfea5404bb,2048,2048,0 3.9748129844665527 eNqtlT9vhSAQwL8StGfj8AYHB...
1,004a270d-34a2-4d60-bbe4-365fca868193,2048,2048,0 0.4322657883167267 eNoLCAhINAgwDMg3NACDBAMHC...
2,00537262-883c-4b37-a3a1-a4931b6faea5,2048,2048,0 0.0999312475323677 eNqNkaEOBCEMRH+p5CoQCAQC0...
3,00c9a1c9-2f06-476f-8b0d-6d01032874a2,2048,2048,0 0.5758232474327087 eNrtVMEKwyAM/aUEhHrY0YOHI...
4,0173029a-161d-40ef-af28-2342915b22fb,3072,3072,0 2.7333269119262695 eNozyEkwCIk3MoADMy/CbGRAj...
...,...,...,...,...
554,fea47298-266a-4cf4-93bd-55d1bcc2fc7d,1728,1728,0 0.34724387526512146 eNp1UNsKwjAM/aVsHStsTKXg...
555,feb955db-6c07-4717-a98b-92236c8e01d8,2048,2048,0 0.42287686467170715 eNqdlMFu2zAMhl/JsrfUAbJb...
556,fefb9bb7-934a-40d1-8d2f-210265857388,2048,2048,0 4.711294651031494 eNoLMEywTEgzNAABHxMDvLRDgg...
557,ff069fa2-d948-408e-91b3-034cfea428d1,3072,3072,0 3.577763557434082 eNrNVcEKgzAM/aUYZk8ePQWqyD...


In [24]:
#df_imlist.to_csv('submission.csv',index = False)

In [25]:

del model, model_4, model_2, model_3, model_g
del model_whole, model_whole_2, model_whole_3 

In [26]:
df = df_imlist

In [27]:
# xmid, ymid / ImageWidth,ImageHeight
for i in range(df.shape[0]):
        a = df.loc[i,'PredictionString']
        b = a.split()
        for j in range(int(len(a.split())/5)):
            c = b[5 * j + 3]
            d = b[5 * j + 4]
            b[5 * j + 3] = str( float(c) / df.loc[i,'ImageWidth'] )
            b[5 * j + 4] = str( float(d) / df.loc[i,'ImageHeight'] )
        df.loc[i,'PredictionString'] = ' '.join(b)

In [28]:
ss_df = df 

In [29]:
debug = False

In [30]:
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(0)
#torch.cuda.empty_cache()


import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import cv2

if debug:
    test_df = pd.read_csv(f'/kaggle/input/hpa-single-cell-image-classification/sample_submission.csv')
    test_df = test_df[:3]
else:
    test_df = pd.read_csv(f'/kaggle/input/hpa-single-cell-image-classification/sample_submission.csv')


test_df['ID'] = test_df['ID'] +'_green'
df = test_df.copy()

save_dir_train = '/kaggle/tmp/0/'
os.makedirs(save_dir_train, exist_ok=True)


# save 512 green png
for i in range(df.shape[0]):
    image_id = df.loc[i,'ID']

    img = cv2.imread(f'/kaggle/input/hpa-single-cell-image-classification/test/{image_id}.png')


    img = cv2.resize(img, (512, 512), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(save_dir_train + image_id +'.png',img)

dim = 512 #1024, 256, 'original'
test_dir = f'/kaggle/tmp/0'
weights_dir = '/kaggle/input/hpa-yolov5-train-all-fine/yolov5/runs/train/exp/weights/last.pt'

shutil.copytree('/kaggle/input/yolov5-official-v31-dataset/yolov5', '/kaggle/working/yolov5')
os.chdir('/kaggle/working/yolov5') # install dependencies

import torch
#from IPython.display import Image, clear_output  # to display images

#clear_output()
#print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))


!python detect.py --weights $weights_dir\
--img 512\
--conf 0.001\
--iou 0.5\
--source $test_dir\
--save-txt --save-conf --exist-ok
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes
image_ids = []
PredictionStrings = []

for file_path in tqdm(glob('runs/detect/exp/labels/*.txt')):
    image_id = file_path.split('/')[-1].split('.')[0]
    w, h = test_df.loc[test_df.ID==image_id,['ImageWidth', 'ImageHeight']].values[0]
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))


pred_df = pd.DataFrame({'ID':image_ids,
                        'PredictionString':PredictionStrings})
print(pred_df)
#shutil.rmtree('/kaggle/working/yolov5')
if debug:
    df1 = pd.read_csv('/kaggle/input/hpa-single-cell-image-classification/sample_submission.csv')
    df1 = df1[:3]
else:
    df1 = pd.read_csv('/kaggle/input/hpa-single-cell-image-classification/sample_submission.csv')
df1 = df1.drop(['PredictionString'], axis=1)
df1['ID'] = df1['ID'] + '_green'
df1 = pd.merge(df1, pred_df, on = 'ID', how = 'left').fillna("0 0 0 0 1 1")

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.001, device='', exist_ok=True, img_size=512, iou_thres=0.5, name='exp', project='runs/detect', save_conf=True, save_txt=True, source='/kaggle/tmp/0', update=False, view_img=False, weights=['/kaggle/input/hpa-yolov5-train-all-fine/yolov5/runs/train/exp/weights/last.pt'])
Fusing layers... 
image 1/559 /kaggle/tmp/0/0040581b-f1f2-4fbe-b043-b6bfea5404bb_green.png: 512x512 33 0. Nucleoplasms, 18 1. Nuclear membranes, 16 2. Nucleolis, 22 3. Nucleoli fibrillar centers, 25 4. Nuclear speckless, 37 5. Nuclear bodiess, 12 6. Endoplasmic reticulums, 24 7. Golgi apparatuss, 5 8. Intermediate filamentss, 3 9. Actin filamentss, 19 12. Centrosomes, 5 13. Plasma membranes, 22 14. Mitochondrias, 1 15. Aggresomes, 36 16. Cytosols, 22 17. Vesicles and punctate cytosolic patternss, Done. (0.050s)
image 2/559 /kaggle/tmp/0/004a270d-34a2-4d60-bbe4-365fca868193_green.png: 512x512 19 0. Nucleoplasms, 13 1. Nuclear membranes, 12 2. Nucleol

  0%|          | 0/559 [00:00<?, ?it/s]

                                             ID  \
0    ff069fa2-d948-408e-91b3-034cfea428d1_green   
1    d267f38c-6235-4c37-8f7a-030801204342_green   
2    12cb3a83-8a33-4bc3-9f7b-6bc096f616b4_green   
3    12667c6f-1d7e-44ca-9b75-ce8b6ddeaa0a_green   
4    f4a7ef21-94ed-4db4-a578-6b1ee1228fe6_green   
..                                          ...   
554  0ea01e42-dee2-46d3-b189-930ff2ac9841_green   
555  9a39e0e1-a1b8-4313-a931-f30bfe246f39_green   
556  84156bfd-d57d-4246-8501-a9b08d49aa85_green   
557  af7d5a9f-285e-4dda-b4ed-92880fa45aa6_green   
558  2ab1f80e-2a07-4785-b7db-249a8c35d23e_green   

                                      PredictionString  
0    16 0.00581360003 1446 240 2076 786 14 0.005855...  
1    0 0.002067570109 0 0 90 90 0 0.002069470007 14...  
2    17 0.001006130013 1212 860 1748 1476 18 0.0010...  
3    15 0.012420699932 880 1488 1444 2048 9 0.01248...  
4    2 0.003902439959 1144 0 1528 352 10 0.00390625...  
..                                           

In [31]:
#yolo output xmid ymid

for i in range(df1.shape[0]):
    list1 = []
    a = df1.loc[i, 'PredictionString'].split()
    for j in range(int(len(a) / 6)):
        list1.append(a[j * 6])
        list1.append(a[j * 6 + 1])
        x_mid = str((int(a[j * 6 + 2]) + int(a[j * 6 + 4])) / 2 / df1.loc[i, 'ImageWidth'])
        y_mid = str((int(a[j * 6 + 3]) + int(a[j * 6 + 5])) / 2 / df1.loc[i, 'ImageWidth'])
        list1.append(x_mid)
        list1.append(y_mid)
    df1.loc[i, 'PredictionString'] = ' '.join(list1)

for i in range(df1.shape[0]):
    df1.loc[i,'ID'] = df1.loc[i,'ID'][:-6]


df1 = df1.sort_values(by=['ID']).reset_index(drop=True)

cuda.select_device(0)
cuda.close()
cuda.select_device(0)


df = ss_df.sort_values(by=['ID']).reset_index(drop=True)
from sklearn.neighbors import KNeighborsClassifier


# get conf=0 df
df2 = df.copy()
for i in range(df2.shape[0]):
        list1 = []
        a = df2.loc[i,'PredictionString']
        b = a.split()
        for j in range(int(len(a.split())/5)):
            list1.append(b[0 + 5 * j])
            list1.append('0')
            list1.append(b[0 + 5 * j +1])
            list1.append(b[0 + 5 * j +2])
            list1.append(b[0 + 5 * j +3])
        df2.loc[i,'PredictionString'] = ' '.join(list1)
#cell xmid ymid ,yolo xmid ymid KNN n=1
for i in range(df.shape[0]):
    list1 = []
    a = df.loc[i,'PredictionString'].split()
    
    for j in range(int(len(df.loc[i, 'PredictionString'].split()) / 19 / 5)):
        list2 = []
        list2.append(float(a[95 * j + 3]))
        list2.append(float(a[95 * j + 4]))
        list1.append(list2)
    list3 = list(range(0,int(len(df.loc[i, 'PredictionString'].split()) / 19 / 5)))
    mm1 = []
    b = df1.loc[i,'PredictionString'].split()
    for j in range(int(len(df1.loc[i, 'PredictionString'].split()) / 4)):
        mm2 = []
        mm2.append(float(b[4 * j +2]))
        mm2.append(float(b[4 * j +3]))
        mm1.append(mm2)
    neigh = KNeighborsClassifier(n_neighbors=1)
    neigh.fit(list1, list3)
    c = neigh.predict(mm1)
    d = df2.loc[i, 'PredictionString'].split()
    e = df1.loc[i, 'PredictionString'].split()
    for j in range(int(len(df1.loc[i, 'PredictionString'].split()) / 4)):
        f = float(d[c[j] * 19 * 5 + int(e[4 * j]) * 5 + 1])
        g = float(e[4 * j + 1])
        # One cell has two confidence values, choose the larger one
        if g > f:
            d[c[j] * 19 * 5 + int(e[4 * j]) * 5 + 1] = str(g) 
    df2.loc[i,'PredictionString'] = ' '.join(d)

#Confidence weighted average
for i in range(df.shape[0]):
    a = df.loc[i,'PredictionString']
    b = a.split()
    df2_str = df2.loc[i,'PredictionString']
    df2_split = df2_str.split()
    for j in range(int(len(a.split())/5)):
        for k in range(19):
            if int(b[0 + 5 * j]) == k:
                c = b[0 + 5 * j + 1]  
                df2_conf = df2_split[0 + 5 * j + 1]
                b[0 + 5 * j + 1] = str(float(c) + float(df2_conf))# * 0.9 + float(c) * 0.1

    df.loc[i,'PredictionString'] = ' '.join(b)
    
# drop xmid ymid
for i in range(df.shape[0]):
    list1 = []
    a = df.loc[i,'PredictionString']
    b = a.split()
    for j in range(int(len(a.split())/5)):
        list1.append(b[0 + 5 * j])
        list1.append(b[0 + 5 * j +1])
        list1.append(b[0 + 5 * j +2])
    df.loc[i,'PredictionString'] = ' '.join(list1)
ss_df = df   


ss_df = ss_df[['ID','ImageWidth','ImageHeight','PredictionString']]
ss_df.to_csv('/kaggle/working/submission.csv',index = False)  
shutil.rmtree('/kaggle/working/yolov5')